In [5]:
# Imports 
import tensorflow as tf
import pandas as pd
import tensorflow_text as text
import nltk
from tensorflow.keras.layers import TextVectorization
import numpy as np
from tensorflow.keras.layers import Embedding


In [6]:
# Load example text to be pre-processed
text_file = open("..\..\data\candide_chapter_1.txt")
data = text_file.read().replace("\n", " ").replace("\"","").replace("[1]","")
text_file.close()

In [7]:
def preprocess(input):
    input = str(input)
    sentences = nltk.sent_tokenize(input)
    formality_model = tf.keras.models.load_model("C:\\Users\\jack-\\Documents\\University\\Project\\src\\deep_learning\\bert_small_e20_b32",compile=False)
    formalities = []
    for sentence in sentences:
        formalities.append(formality_model.predict([sentence])[0][0])
    formalities.sort()
    output = []
    output.append(formalities[:5])
    output.append(formalities[-5:])
    flat_list = [item for sublist in output for item in sublist]
    return flat_list

In [9]:
vectorizer = TextVectorization(max_tokens=20000, output_sequence_length=200)
text_ds = tf.data.Dataset.from_tensor_slices([data]).batch(128)
vectorizer.adapt(text_ds)

voc = vectorizer.get_vocabulary()
word_index = dict(zip(voc, range(len(voc))))

path_to_glove_file = "..\..\src\deep_learning\glove.6B.100d.txt"

embeddings_index = {}
with open(path_to_glove_file,encoding="utf-8") as f:
    for line in f:
        word, coefs = line.split(maxsplit=1)
        coefs = np.fromstring(coefs, "f", sep=" ")
        embeddings_index[word] = coefs

print("Found %s word vectors." % len(embeddings_index))

num_tokens = len(voc) + 2
embedding_dim = 100
hits = 0
misses = 0

# Prepare embedding matrix
embedding_matrix = np.zeros((num_tokens, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # Words not found in embedding index will be all-zeros.
        # This includes the representation for "padding" and "OOV"
        embedding_matrix[i] = embedding_vector
        hits += 1
    else:
        misses += 1
print("Converted %d words (%d misses)" % (hits, misses))

embedding_layer = Embedding(
    num_tokens,
    embedding_dim,
    embeddings_initializer=tf.keras.initializers.Constant(embedding_matrix),
    trainable=False,
)

Found 400000 word vectors.
Converted 336 words (13 misses)


In [11]:
# Create model
int_sequences_input = tf.keras.Input(shape=(None,), dtype="int64")
embedded_sequences = embedding_layer(int_sequences_input)
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(embedded_sequences)
x = tf.keras.layers.MaxPooling1D(1)(x)
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tf.keras.layers.MaxPooling1D(1)(x)
x = tf.keras.layers.Conv1D(128, 1, activation="relu")(x)
x = tf.keras.layers.GlobalMaxPooling1D()(x)
x = tf.keras.layers.Dense(128, activation="relu")(x)
x = tf.keras.layers.Dropout(0.5)(x)
preds = tf.keras.layers.Dense(units=2)(x)
model = tf.keras.Model(int_sequences_input, preds)
model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, None)]            0         
_________________________________________________________________
embedding (Embedding)        (None, None, 100)         35100     
_________________________________________________________________
conv1d_3 (Conv1D)            (None, None, 128)         12928     
_________________________________________________________________
max_pooling1d_2 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_4 (Conv1D)            (None, None, 128)         16512     
_________________________________________________________________
max_pooling1d_3 (MaxPooling1 (None, None, 128)         0         
_________________________________________________________________
conv1d_5 (Conv1D)            (None, None, 128)         1651

In [15]:
train = vectorizer(np.array([[s] for s in [data]])).numpy()
model.compile(
    loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["acc"]
)
model.fit(train, [1], batch_size=128, epochs=20)


ValueError: Failed to find data adapter that can handle input: <class 'numpy.ndarray'>, (<class 'list'> containing values of types {"<class 'int'>"})